In [1]:
import os

# 判断是否在远程服务器环境
if os.path.exists("/home/ubuntu/data/pycharm_project_377"):
# 远程服务器环境
    os.chdir("/home/ubuntu/data/pycharm_project_377/GPU_32_pythonProject")
    print(os.getcwd())
else:
# 本地环境
#     os.chdir("/Users/coyi/PycharmProjects/coyi_pythonProject/")
    os.chdir("/Users/coyi/PycharmProjects/coyi_pythonProject/RAG/")
    base_dir="/Users/coyi/PycharmProjects/coyi_pythonProject/RAG/"
    print(os.getcwd())

/home/ubuntu/data/pycharm_project_377/GPU_32_pythonProject


In [2]:
# 设置openai 代理
import os
os.environ["OPENAI_API_BASE"] = "https://api.openai-proxy.org/v1" # 这是代理，如果你是官方key ，就不要设置这个

os.environ["OPENAI_API_KEY"]   = "sk-CQI7eRa8J8QBW94nmlSWplqsWqvz6ZwSJ3Xkw1FO9vE2DcUh" # 改成自己的key

In [3]:
import os

# 指定 Hugging Face 镜像源代理
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

# # 如果使用 datasets 包，开启离线模式
# os.environ["HF_DATASETS_OFFLINE"] = "1"
#
# # 如果使用 evaluate/metrics，开启离线模式
# os.environ["HF_METRICS_OFFLINE"] = "1"

In [4]:
# # Milvus 向量数据库 - 服务器版示例 (Milvus 2.5)
# # 安装依赖：
# # pip install -U "langchain-milvus>=0.1.4" "pymilvus>=2.4.3" "sentence-transformers"
#
# import os, pathlib, time, hashlib, logging
# from typing import List
#
# from langchain_huggingface.embeddings import HuggingFaceEmbeddings
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document
# from langchain_milvus import Milvus          # ← 官方推荐实现
#
# # 禁用 tokenizers 并设置日志级别为 INFO
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# logging.basicConfig(level=logging.INFO)
# from huggingface_hub import HfApi
# api = HfApi(endpoint="https://hf-mirror.com")
#
# # ── 0. 路径 & 环境 ────────────────────────────────────────────
# # 以下两行保留，仅为兼容原有逻辑创建本地目录（实际服务器模式下无需使用此目录）
# BASE = pathlib.Path.cwd() / "milvus_lite" / "prod"
# BASE.mkdir(parents=True, exist_ok=True)
# # 本地 Lite 模式的 URI 保留，但下文不再使用
# URI = str(BASE / "lite.db")
# print("Local URI (保留):", URI)
#
# # 指定 Milvus 服务器连接参数：
# #   host: Milvus server IP 或域名
# #   port: Milvus gRPC 端口（默认 19530）
# MILVUS_CONFIG = {"host": "62.234.111.133", "port": 19530}
# COL = "hybrid_rag"
#
# # ── 1. 生成向量 ──────────────────────────────────────────────
# # 初始化向量模型和文本切分器（与本地模式一致）
# embed = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-large-zh-v1.5",  # 向量模型名称
#     cache_folder="/home/ubuntu/models"    # 本地缓存路径
# )
# split = RecursiveCharacterTextSplitter(
#     chunk_size=512,                         # 每个文本块最大长度
#     chunk_overlap=64                        # 文本块重叠长度
# )
#
# # texts_source 需提前定义为待索引文本列表
# # 构建文档列表和对应主键
# docs, ids = [], []
# for i, raw in enumerate(texts_source, 1):
#     for chunk in split.split_text(str(raw)):
#         # 生成内容指纹作为主键
#         pk = hashlib.md5(chunk.encode()).hexdigest()
#         # 添加元数据字段（文档ID和插入时间戳）
#         meta = {"biz_id": f"doc_{i}", "inserted_at": int(time.time())}
#         docs.append(Document(page_content=chunk, metadata=meta))
#         ids.append(pk)
#
# # ── 2. 首次初始化 ────────────────────────────────────────────
# # 如果集合不存在，则从文档批量创建
# # 使用服务器版连接参数替换本地 URI
# from_documents_args = {
#     "documents": docs,
#     "ids": ids,
#     "embedding": embed,
#     "collection_name": COL,
#     "connection_args": MILVUS_CONFIG,         # ★ 服务器模式 ★
#     "index_params": {"index_type": "IVF_FLAT", "metric_type": "L2"},
#     "auto_id": False,
# }
# # 执行首次写入
# Milvus.from_documents(**from_documents_args)
# logging.info("INIT ➜ 写入 %s 向量", len(ids))
#
# # ── 3. 增量同步 (幂等) ────────────────────────────────────────
# # 构建服务器版客户端实例
# vs = Milvus(
#     embedding_function=embed,
#     collection_name=COL,
#     connection_args=MILVUS_CONFIG,             # ★ 服务器模式 ★
#     auto_id=False,
# )
#
# # ❶ 删除待同步主键（即使不存在也无错误）
# vs.delete(ids=ids)
# # ❷ 再一次性插入最新版本，实现 Upsert 效果
# vs.add_documents(docs, ids=ids)
#
# logging.info("SYNC ➜ 完成 %s 条 Upsert", len(ids))

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
milvus_rag_hybrid.py  ——  电商领域混合检索 / 问答 Demo
=====================================================
核心特点
---------
1. **向量侧**：BGE‑large‑zh‑v1.5 生成稠密向量，Milvus 2.5 里建 **HNSW** 索引。
2. **稀疏侧**：Milvus 2.5 「全文检索」功能，自动把 text → sparse_vector（**BM25**）。
3. **混合召回**：向量  +  BM25  两路，各取 *k* 条合并去重。
4. **重排**      ：BGE reranker（`BGERerankFunction`）打分挑前 *top_n*。
5. **答案生成**：在线 GPT‑4（可改本地 ChatGLM3）生成最终回复。
6. **增量入库**：`ingest <folder>` 读取文件夹中 *.txt*，已存在（MD5 主键）则跳过。

运行前准备
-----------
::
    # ①  部署 Milvus 2.5 Stand‑alone（Docker）
    docker run -d --name milvus25 \
      -p 19530:19530 -p 9091:9091 \
      -e TZ=Asia/Shanghai \
      -v $PWD/milvus_data:/var/lib/milvus \
      milvusdb/milvus:v2.5.1

    # ②  Python 依赖
    python -m pip install -U \
        pymilvus==2.5.* pymilvus-model \
        sentence-transformers \
        openai
    # ✅ 模型权重( BGE / reranker )提前下载可加速首次运行

用法
----
::
    # 批量导入 ./docs 目录下的 txt 文件
    python milvus_rag_hybrid.py ingest ./docs

    # 交互式提问
    python milvus_rag_hybrid.py ask "推荐引擎常用算法有哪些？"
"""

import argparse
import getpass
import sys
from pathlib import Path
from typing import List, Dict
import os, time, hashlib, logging, torch
# 新版（>=1.0.0）推荐：
from openai import OpenAI
from pymilvus import (
    MilvusClient, FieldSchema, CollectionSchema, DataType, Function, FunctionType, Collection,connections,utility
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction
from pymilvus.model.reranker import BGERerankFunction
from sentence_transformers import models  # 仅用于提前下载权重，可选


In [6]:
# ============================================================================
# 全局配置 —— 根据需要修改
# ============================================================================
MILVUS_URI = os.getenv("MILVUS_URI", "127.0.0.1:19530")
COLLECTION_NAME = "ecommerce_rag"

# —— 嵌入 / 重排 模型
EMBED_MODEL_NAME = "BAAI/bge-large-zh-v1.5"
RERANK_MODEL_NAME = "BAAI/bge-reranker-v2-m3"
# —— OpenAI
## 启动脚本时，若环境没设，就弹一次输入
# 全局openai client，初次用时由 ensure_api_key 初始化
# 先在 shell 中 export OPENAI_API_KEY=sk-xxx
OPENAI_MODEL = "gpt-4.1-mini-2025-04-14"
client: OpenAI | None = None  # 稍后由 ensure_client() 初始化
# —— 检索 / 重排 超参
TOP_K_VECTOR = 50
TOP_K_BM25 = 100
TOP_N_RERANK = 20
MAX_HISTORY = 2  # 最近对话轮数拼进 prompt，避免超长

In [7]:
# —— 第一步：建立默认连接 alias
# 从全局配置的 MILVUS_URI 中解析 host 和 port
host, port = MILVUS_URI.replace("http://", "").replace("https://", "").split(":")
connections.connect(
    alias="default",
    host=host,
    port=port,
)

In [13]:
# 删除模块，谨慎放开～～～
from pymilvus import utility

# 1. 确认连接别名和 COLLECTION_NAME
ALIAS = "default"
COLLECTION_NAME = "ecommerce_rag"

# 2. 删除整个 Collection（连同它上面的数据和索引都会被清除）
if COLLECTION_NAME in utility.list_collections(using=ALIAS):
    utility.drop_collection(COLLECTION_NAME, using=ALIAS)
    print(f"✅ 已删除 Collection `{COLLECTION_NAME}`")
else:
    print(f"⚠️ Collection `{COLLECTION_NAME}` 不存在，跳过删除")



⚠️ Collection `ecommerce_rag` 不存在，跳过删除


In [9]:
# =============================================================================
# Milvus — Collection 保证存在（一次性）
# =============================================================================

def ensure_collection() -> Collection:
    """在 Milvus 中创建（或连接）名为 COLLECTION_NAME 的集合。"""
    # 1. 建立到 Milvus 服务的连接


    # 2. 查询当前已有的集合列表，如果 COLLECTION_NAME 已存在，则直接返回对应的 Collection 对象
    #    list_collections() 返回一个字符串列表，每个元素都是已创建集合的名称
    if COLLECTION_NAME in utility.list_collections(using="default"):
        # Collection() 接受集合名称，封装后可以执行 insert/search 等操作
        return Collection(COLLECTION_NAME, using="default")

    # ——————————————————————————————————————————————————————————————————————
    # 3. 如果集合不存在，下面开始构造它的 Schema（集合结构）并创建
    # ——————————————————————————————————————————————————————————————————————

    # 3.1 定义各个字段（Field）
    fields = [
        # 文档主键——用 MD5 指纹字符串做唯一标识
        FieldSchema(
            name="doc_id",                       # 字段名
            dtype=DataType.VARCHAR,              # 字符串类型
            max_length=32,                       # 最长 32 字符
            is_primary=True,                     # 主键
            auto_id=False                        # 不自动生成，需要外部提供
        ),
        # 原始文本字段，用于 BM25 全文检索
        FieldSchema(
            name="text",
            dtype=DataType.VARCHAR,
            max_length=65535,                    # 支持大块文本
            enable_analyzer=True                 # 开启分词器，支持 BM25 全文检索
        ),
        # 稠密向量字段（Dense Vector），用于近似向量搜索
        FieldSchema(
            name="dense",
            dtype=DataType.FLOAT_VECTOR,         # 浮点向量
            dim=1024                             # 向量维度，与 BGE-large-zh-v1.5 输出的向量维度保持一致
        ),
        # 稀疏向量字段（Sparse Vector），通过内置 BM25 算子生成
        FieldSchema(
            name="sparse",
            dtype=DataType.SPARSE_FLOAT_VECTOR,  # 稀疏向量类型
            is_function=True                     # 表明此字段由 Function 生成 # 稀疏向量由 BM25 算子自动生成
        ),
    ]

    # 3.2 构造 CollectionSchema对象，附带描述信息
    schema = CollectionSchema(fields, description="电商项目混合检索 RAG Collection")

    # --- 注册 BM25 函数算子 ---
    # 3.3 # BM25 函数：text -> sparse
    bm25_fn = Function(
        name="bm25_gen",                         # 算子名称，可随意命名
        input_field_names=["text"],              # 输入字段：原始文本
        output_field_names=["sparse"],           # 输出字段：稀疏向量
        function_type=FunctionType.BM25,         # 算子类型：BM25 全文检索
    )
    # 将算子注册到 schema 中
    schema.add_function(bm25_fn)

    # 4. 创建集合：传入集合名和完整的 schema
    # （必须显式指定 schema）：
    # # 直接 new 一个 Collection，会在服务端自动创建（若不存在）：
    collection = Collection(
        name=COLLECTION_NAME,
        schema=schema,
        using="default" # 指定链接alias "default" 下操作
    )

    # ——————————————————————————————————————————————————————————————————————
    # 5. 创建索引：分别对稠密和稀疏字段建立索引，以加速搜索
    # ——————————————————————————————————————————————————————————————————————

    # --- HNSW 索引（用于 dense 向量检索） ---
    #   • M：最大邻居数，值越大召回率越高但构建/查询速度越慢
    #   • efConstruction：构建阶段的候选列表大小，越大质量越高但耗时越多
    # 5.1 对 dense 向量字段创建 HNSW 索引（高性能近似搜索）
    collection.create_index(
        field_name="dense",                                # 索引字段
        index_params={
            "index_type": "HNSW",               # 索引算法
            "metric_type": "COSINE",
            "params": {"M": 16, "efConstruction": 200}
        }
    )


    # --- 倒排索引（用于 sparse BM25 检索） ---
    #   • SPARSE_INVERTED_INDEX：Milvus 内置的稀疏倒排结构
    #   • metric_type="BM25"：采用 BM25 打分机制
    # 5.2 对 sparse 字段创建倒排索引，使用 BM25 距离度量
    collection.create_index(
        field_name="sparse",
        index_params={
            "index_type": "SPARSE_INVERTED_INDEX",
            "params": {"metric_type": "BM25"}
        }
    )
    # 接下来所有的索引操作，都用 col.create_index()
    idx_info = collection.indexes  # ORM 属性，查看已创建的索引
    if "dense" not in {i.field_name for i in idx_info}:
        print("❌ dense 创建失败")
    else:
        print("✅ dense 创建完成")
    if "sparse" not in {i.field_name for i in idx_info}:
        print("❌ sparse 创建失败")
    else:
        print("✅ sparse 创建完成")
    # 打印提示：集合创建完毕
    print("✅ Collection 创建完成")

    # 返回新创建的 Collection 对象，后续可直接用它进行插入/查询
    return Collection(COLLECTION_NAME, using="default")

In [10]:
# =============================================================================
# 初始化单例 —— 嵌入 & 重排函数初始化 rerank
# =============================================================================

# 6. 准备好向量化函数：SentenceTransformerEmbeddingFunction
#    将文本批量转化为稠密向量，device 根据是否有 GPU 自动选择
EMBED_FN = SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL_NAME,
    cache_folder="/home/ubuntu/models",                  # ← 新增：本地模型缓存路径
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# 7. 准备好重排函数：BGE Reranker
#    用于对初筛结果进行进一步排序
RERANK_FN = BGERerankFunction(
    model_name=RERANK_MODEL_NAME,
    cache_folder="/home/ubuntu/models",                  # ← 新增：本地模型缓存路径
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# 8. 创建或获取集合——并赋值给全局变量 COL，供后续代码使用 获取 Collection，并加载
COL = ensure_collection()
COL.load()

✅ dense 创建完成
✅ sparse 创建完成
✅ Collection 创建完成


In [11]:
def ingest_folder(folder: str, batch_size: int = 1000):
    """
    把 folder 下所有 .txt 文件切块后写入 Milvus（按 MD5 去重）。

    改进点：
    1. 批量去重：一次性查询所有新计算的 doc_id，而不是每块单独查询。
    2. 分批插入：按 batch_size 分批 insert + flush，避免一次性压力过大。
    3. 错误处理：捕获关键步骤异常，记录日志并跳过失败部分。
    """
    # 将传入的路径字符串转为 Path 对象，方便做文件系统操作
    folder_path = Path(folder)
    # 如果目录不存在，提前返回
    if not folder_path.exists():
        print(f"❌ 目录不存在: {folder}")
        return

    # 文本切分器：每块 512 字符，重叠 64 字符，保证上下文连贯
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

    # 1. 遍历目录下所有 .txt 文件，把每个文件切分后的块存到 all_chunks 列表
    all_chunks: List[Dict] = []
    for txt_file in folder_path.rglob("*.txt"):
        # 读取文件内容，忽略编码错误
        raw = txt_file.read_text(encoding="utf-8", errors="ignore")
        # 把整个文本切成多个 chunk
        for chunk in splitter.split_text(raw):
            # 用 chunk 的 MD5 作为唯一主键，后面用于去重
            pk = hashlib.md5(chunk.encode()).hexdigest()
            # 存成字典，后面统一添加向量字段并插入
            all_chunks.append({"doc_id": pk, "text": chunk})

    # 如果没有读到任何文本块，说明没有 .txt 或都为空
    if not all_chunks:
        print("⚠️ 无 .txt 文件或文件为空")
        return

    # 2. 批量去重 —— 构造一个 Milvus 表达式，查询已有的 doc_id
    all_pks = [e["doc_id"] for e in all_chunks]
    # 构造表达式："doc_id in ['id1','id2',...]"
    expr = "doc_id in [" + ",".join(f"'{pk}'" for pk in all_pks) + "]"
    try:
        existing = COL.query(expr=expr, output_fields=["doc_id"])
        existing_pks = {r["doc_id"] for r in existing}
    except Exception as e:
        # 如果查询失败，就跳过去重，全部当作新数据插入
        print("⚠️ 去重查询失败，跳过去重", e)
        existing_pks = set()

    # 3. 筛选出真正需要插入的文本块
    new_entities = [e for e in all_chunks if e["doc_id"] not in existing_pks]
    if not new_entities:
        print("⚠️ 无新增内容")
        return

    # 4. 分批处理新数据：一方面生成向量，一方面插入 Milvus
    for i in range(0, len(new_entities), batch_size):
        # 取出当前批次
        batch = new_entities[i : i + batch_size]
        texts = [e["text"] for e in batch]

        # 4.1 向量生成：调用 EMBED_FN，可能抛错
        try:
            vecs = EMBED_FN(texts)
        except Exception as e:
            print(f"❌ 向量生成失败 (batch {i//batch_size}), 跳过该批次: {e}")
            continue

        # 把生成的 dense 向量添加到每个实体字典中
        # batch 是一个列表，里面每个元素都是一个字典，初始包含了 "doc_id" 和 "text" 两个字段
        # 最终执行完后；batch 里的每个实体字典就变成了 { "doc_id": ..., "text": ..., "dense": [...] }，正好符合 Milvus 插入 API 要求。
        for e, v in zip(batch, vecs):
            e["dense"] = v.tolist()  # 转成普通 list，方便 JSON 序列化

        # 4.2 插入 Milvus，并立即 flush(落盘+可查询)
        try:
            COL.insert(batch)
            COL.flush()
        except Exception as e:
            print(f"❌ 插入/刷新失败 (batch {i//batch_size}), 跳过该批次: {e}")
            continue

        print(f"✅ 成功写入 batch {i//batch_size}，共 {len(batch)} 条")

    # 全部批次处理完毕，打印总量
    print(f"🎉 全部完成，新增总计 {len(new_entities)} 条 chunk")
ingest_folder('/home/ubuntu/data/pycharm_project_377/GPU_32_pythonProject/LLM/重排rag/test_rag')

KeyboardInterrupt: 

In [12]:
def ensure_client():
    """
    确保全局 client 已初始化：
      1) 如果环境变量里已有 OPENAI_API_KEY，直接用它
      2) 否则通过 getpass 询问一次，并写回环境变量
    之后创建或更新全局 OpenAI(client)
    """
    global client
    key = os.getenv("OPENAI_API_KEY")
    # 代理地址
    base_url = os.getenv("OPENAI_API_BASE", "https://api.openai-proxy.org/v1")
    if not key:
        # 第一次才提示输入
        print(f"❌ 没有检测到openai key")
        key = getpass.getpass("Enter your OpenAI API key: ").strip()
        os.environ["OPENAI_API_KEY"] = key
    # 初始化新版 v1 客户端
    client = OpenAI(api_key=key,base_url=base_url)
    print(f"✅ 成功初始化client !")

In [13]:
def ask(question: str, history: List[str]):
    global OPENAI_API_KEY   # ← 这一行非常关键 在函数最开头声明
    """
    混合检索 + BGE 重排 + GPT-4.1-mini 生成回答。
    history 为之前的用户问题列表。
    如果没有环境变量中的 API Key，会在第一次调用时交互式提示输入一次，
    并在后续调用时自动复用，不再重复询问。
    """

    # —— ① 两路召回 ——
    # 把用户的 query 转成稠密向量 q_vec（长度为 1024）用于向量检索
    q_vec = EMBED_FN([question])[0]

    # 用 Milvus 的向量检索接口，从 “dense” 字段中召回 TOP_K_VECTOR 条最相似的向量文档
    # metric_type=“COSINE”：指定用「余弦相似度」作为向量之间的距离度量。
    # params={“ef”: TOP_K_VECTOR}：这是 HNSW 索引内部的搜索参数——ef（exploration factor）。
    # ef 越大，搜索时会探索更多候选节点，召回率更高但速度稍慢；通常把它设为与想要返回的 K 值相同可以在效率和召回之间取得平衡。
    vec_hits = COL.search(
        data=[q_vec],
        anns_field="dense",
        param={"metric_type": "COSINE", "params": {"ef": TOP_K_VECTOR}},
        limit=TOP_K_VECTOR,
        output_fields=["text"]           # ← 把 text 字段带回来
    )

    # 用 Milvus 的 BM25 全文检索接口，从 “sparse” 字段中召回 TOP_K_BM25 条最相关的文档
    bm25_hits = COL.search(
        data=[question],
        anns_field="sparse",
        param={},                  # ← 这里不能省！
        limit=TOP_K_BM25,
        output_fields=["text"]           # ← 把 text 字段带回来
    )

    # 将两路召回的结果合并去重（用 doc_id 做 key），只保留 text 字段
    # python 字典key 必须唯一：遇到重复的 doc_id 会自动只保留最后一次出现的那条
    # vec_hits[0] 稠密向量召回 和 bm25_hits[0]BM25 稀疏召回 都是列表，存放这一轮搜索（对单个 query）的所有命中结果
    # 每个 hit 都有 hit.entity.doc_id 和 hit.entity.text 两个属性
    pool = {
        hit.entity.doc_id: hit.entity.text
        for hit in vec_hits[0] + bm25_hits[0]
    }
    texts = list(pool.values())

    # 如果两路召回都没有命中任何文档，就直接返回提示
    if not texts:
        return "(未检索到相关文档)"

    # —— ② 重排 ——
    # 调用 RERANK_FN，对召回的文档 text 列表按照 question 做进一步打分排序
    # rerank_res 是一个打分对象列表，每个元素包含 text 和对应得分
    rerank_res = RERANK_FN(question, texts)

    # 只取重排后的前 TOP_N_RERANK 条文本，拼成 prompt 的上下文
    top_ctx = "\n\n".join(r.text for r in rerank_res[:TOP_N_RERANK])

    # —— ③ Prompt 构造 ——
    # 只保留历史对话的最后 MAX_HISTORY 轮，用于拼接进 prompt
    #Q1:我想买手机
    #Q2:推荐哪些型号？
    short_hist = history[-MAX_HISTORY:]
    # 把历史对话格式化成 Q1:xxx\nQ2:xxx 的形式
    hist_part = "\n".join(f"Q{i+1}:{q}" for i, q in enumerate(short_hist))

    # —— ① System：角色 + 输出规范 ——
    system_prompt = (
        "你是一位中国古典文学专家，精通《红楼梦》《三国演义》《水浒传》《西游记》。\n"
        "请使用中文回答，必须严格遵守以下格式：\n\n"
        "1. 回答主体（分点编号①②③……），每点末尾都要加“引用：[n]”。\n"
        "2. 回答结束后，附“参考文献”区块，列出每个[n]对应的书名、回目/章节和原文示例。\n"
        "3. 若有写作示例需求，放在参考文献之后的“写作示例”区块。\n\n"
        "—— Few-Shot 示例 ——\n"
        "Q: 草船借箭发生在什么历史背景？\n"
        "A:\n"
        "① 时间与背景：三国赤壁之战前夕，曹操大军南下，与东吴形成对峙。[1]\n"
        "② 谋略核心：诸葛亮利用大雾与草船，从曹军处“借”得十万支箭。[2]\n"
        "③ 意义与启示：体现了以逸待劳和心理战的重要性。[3]\n\n"
        "参考文献：\n"
        "[1] 《三国演义》第五十回“苦战七日”：\n"
            "回目概述：本回讲述曹操率大军南征，沿江列营，与孙权、刘备联军对峙；诸葛亮被周瑜试探，并提出三日造箭之难。  \n"
            "原文摘录：\n"
            "“操军于柴桑下江，屯于北岸。……操知江水流急，乃筑浮桥渡营；又恐天寒失粮，乃使人索草于江上为蓑，为衣，以备寒暑。忽闻江北烽起，乃晓为联军已至。操乃令牙门将孟坦镇守营门，亲率数万人登舟南下。未到对岸，只见对岸旌旗蔽日，火焰冲天响震山谷，万箭齐发射来，箭如雨下，船舷尽被射满。” \n"

        "[2] 《三国演义》第五十一回“草船借箭”：\n"
            "回目概述：周瑜设疑，令诸葛亮三日内造十万支箭；诸葛亮运用大雾及草船计，借得曹营箭矢，答标难题。\n"
            "原文摘录：  \n"
            "“孔明笑曰：‘军师勿忧，吾有妙计。请借吾二十丈舸，每舸置草人百余，紧束青布为幔，分布两舷。三更时分，朔风将息，雾气弥漫，彼必疑有人渡江，箭手必向江面乱射。届时收船，箭自满载。’……果然大雾四起，江面茫茫，曹军见状，弓弩齐开，箭如雨降。明教人命舟回，满载箭矢不下十万。” \n"

        "[3] 《三国演义》第五十二回“舌战群儒”：\n"
            "回目概述：赤壁鏖战前，诸葛亮于孙权帐下巧辩东吴群儒，论说联盟利害，力促孙刘同心。场面气氛紧张，为后续赤壁之战奠定舆论基础。\n"
            "原文摘录：\n"
            "“黄盖取酒献与孔明，孔明起面前，手指众儒曰：‘天地不仁，以万物为刍狗；匹夫取死，以国家为儿戏。今操挟天子以征四方，威赫中原；尔曹犹在此彷徨，何不速与联军合纵？’……儒生皆默，其声震彻当场，孙权拍案称善。” \n\n"
    )

    # —— ② Assistant：注入上下文 ——
    assistant_context = (
        "【检索上下文】\n"
        f"{top_ctx}\n\n"
        "【对话历史】\n"
        f"{hist_part}\n\n"
        "——\n\n"
        "请基于以上内容，用“①②③”格式回答用户的当前问题，"
        "并务必在每点后加“引用：[n]”，最后给出“参考文献”列表："
    )

    # —— ③ User：当前问题 ——
    user_prompt = question

    # —— ④ GPT-4 API ——
    # 你的代理和 API Key 已经通过环境变量设置好
    # —— 0. 确保 openai client 就绪 ——
    if client is None:
        ensure_client()

    # 调用 OpenAI ChatCompletion 接口，传入刚才构造的 prompt
    # temperature=0.3 降低随机性，让回答更稳定
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system",    "content": system_prompt},
            {"role": "assistant", "content": assistant_context},
            {"role": "user",      "content": user_prompt},
        ],
        temperature=0.3,
    )

    # 从 API 返回结果中取出生成的内容并去除多余空白后返回
    return response.choices[0].message.content.strip()

In [14]:
def main():
    # 创建一个命令行参数解析器，description 用于在帮助信息中显示脚本功能
    parser = argparse.ArgumentParser(description="Milvus Hybrid RAG (中国古典文学助手)")
    # 添加子命令支持，dest="cmd" 表示解析后将子命令名称存入 args.cmd
    sub = parser.add_subparsers(dest="cmd")

    # —— ingest 子命令 —— 用于导入本地文本文件夹
    ing = sub.add_parser("ingest", help="导入本地文件夹")
    # ingest 命令要接收一个位置参数 folder，表示包含 .txt 的目录路径
    ing.add_argument("folder", help="包含 .txt 的目录")

    # —— ask 子命令 —— 交互式提问模式
    ask_p = sub.add_parser("ask", help="提问模式 (交互式)")
    # question 参数可选，若提供则视为“首个问题”，否则直接进入交互式
    ask_p.add_argument("question", nargs="?", help="首个问题，可空进入 interactive")

    # 解析命令行参数
    # args = parser.parse_args()
    # 改成这一行，忽略未知参数
    args, _ = parser.parse_known_args()

    # 如果用户输入的是 ingest 子命令
    if args.cmd == "ingest":
        # 调用 ingest_folder 函数，将指定目录中的文本切分并写入 Milvus
        ingest_folder(args.folder)
        return  # 处理完毕后退出 main

    # 如果用户输入的是 ask 子命令
    if args.cmd == "ask":
        history: List[str] = []  # 用来保存用户提问的历史列表
        # 如果在命令行中给了 question，就先回答这一条
        if args.question:
            print(ask(args.question, history))
            # 将这次提问加入历史，下次生成 prompt 时会用到
            history.append(args.question)
        # 进入 REPL（Read–Eval–Print Loop），持续读取用户输入
        try:
            while True:
                # 提示用户输入
                q = input("你: ").strip()
                # 如果输入为空，忽略并重新提示
                if not q:
                    continue
                # 调用 ask() 进行检索、重排、生成回答
                ans = ask(q, history)
                # 打印 AI 回答
                print("AI:", ans)
                # 保存这次提问到历史
                history.append(q)
        # 捕获用户按 Ctrl+D 或 Ctrl+C 的退出操作
        except (EOFError, KeyboardInterrupt):
            print("\n退出对话")
        return  # 退出 main

    # 如果既不是 ingest 也不是 ask，则显示帮助信息
    parser.print_help()


# Python 脚本的标准写法：只有当直接运行此文件时，才调用 main()
if __name__ == "__main__":
    main()

In [15]:
print(ask("孙悟空为什么打白骨精？？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  5.47it/s]


✅ 成功初始化client !
① 白骨精变作孙悟空模样，冒充孙悟空行恶，欺骗唐僧，导致唐僧误会孙悟空，孙悟空为保护师父和取经队伍的安全，必须除掉这个妖精以防祸害。[1]

② 白骨精先后三次变化，企图掳走唐僧，孙悟空识破其伎俩，多次出手打杀妖精，维护取经的正道和师徒安全，体现了孙悟空的机智和忠诚。[1]

③ 孙悟空打白骨精虽被唐僧误解为滥杀无辜，但实际上是为了保护师父和取经大业，显示出孙悟空对师父的忠心和对妖魔的坚决斗争态度。[1]

参考文献：  
[1] 《西游记》第五十七回“孙行者三打白骨精”：  
回目概述：白骨精三次变化欺骗唐僧，孙悟空识破并三次打杀妖精，虽被唐僧误解，但最终揭穿妖精真面目。  
原文摘录：  
“我因保护唐僧往西天取经在路上打杀贼徒，那三藏赶我回去，我径到普陀崖见观音菩萨诉苦，不想这妖精几时就变作我的模样，打倒唐僧抢去包袱……原来这妖精果象老孙模样，才自水帘洞打到普陀山见菩萨，菩萨也难识认，故打至此间烦诸天眼力与我认个真假。”


In [16]:
print(ask("贾宝玉与林黛玉的爱情悲剧反映了怎样的社会现实与家族压力？？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  3.16it/s]


① 家族利益与婚姻安排的冲突：贾宝玉与林黛玉的爱情被家族的利益和婚姻安排所左右。薛宝钗的出现，作为家族认可的门当户对的婚配对象，体现了封建家族对婚姻的功利性考量，个人感情往往被牺牲。[1]

② 社会身份与礼教束缚：宝玉虽性情纯真，但身处封建礼教严格的社会环境中，必须遵守家族的规矩和社会的等级秩序。宝玉的懒散和反叛，实际上是对这种束缚的无声抗议，但最终难以逃脱命运的安排。[2]

③ 女性命运的悲剧体现：林黛玉的多愁善感和体弱多病，象征着封建社会中女性的脆弱与无奈。她在家族和社会的双重压力下，无法实现自我价值和幸福，最终成为爱情悲剧的牺牲品。[3]

④ 家族内部矛盾与情感疏离：贾府内部虽表面繁华，但亲情关系复杂，宝玉与黛玉的亲密也夹杂着嫉妒和误解，反映了封建家族内部的矛盾和情感的疏离，增加了悲剧的深度。[4]

参考文献：  
[1] 《红楼梦》第五回“游幻境指迷十二钗”及相关章节：薛宝钗的入府及其与黛玉的对比，家族对婚姻的安排。  
[2] 《红楼梦》第七回“送宫花贾琏戏熙凤”及相关描写：宝玉的性格与家族规矩的冲突。  
[3] 《红楼梦》第十三回“秦可卿死封龙禁尉”及相关章节：黛玉的体弱多病和女性命运的描写。  
[4] 《红楼梦》第三回“贾宝玉神游太虚境”及相关章节：贾府内部复杂的人际关系和情感纠葛。


In [17]:
print(ask("宋江在梁山好汉中被称作什么绰号？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  2.79it/s]


① 宋江在梁山好汉中被称为“及时雨”。这一绰号源于他乐于助人、行侠仗义的性格，像春雨及时滋润大地，帮助有难的人。[1]

② 此外，宋江也被称为“呼保义”，意为“号召义气”，体现了他在梁山好汉中的领导地位和号召力。[2]

③ 在《水浒传》中，宋江的绰号多次被提及，显示了他在江湖上的声望和人缘，尤其是在梁山泊聚义时，众好汉对他的尊敬。[3]

参考文献：  
[1] 《水浒传》第三回“及时雨宋江杀惜”  
回目概述：介绍宋江的绰号“及时雨”及其乐善好施的性格。  
原文摘录：“宋江，字公明，郓城县郓城县人氏，绰号‘及时雨’，因他遇事及时相助，故得此名。”  

[2] 《水浒传》第四回“林教头风雪山神庙”  
回目概述：宋江被称为“呼保义”，意指号召义气，表现其领导才能。  
原文摘录：“呼保义宋江，义气深重，众好汉皆敬仰之。”  

[3] 《水浒传》第七回“柴进小聚义”  
回目概述：梁山好汉聚义，众人称宋江为“及时雨”，彰显其威望。  
原文摘录：“众好汉见宋江，齐声呼作‘及时雨’，皆感其恩德。”


In [18]:
print(ask("草船借箭”发生在什么历史背景和军事对峙中？？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  2.87it/s]


① 时间背景：草船借箭发生在三国时期，赤壁之战前夕，曹操率大军南下，与孙权、刘备联军形成对峙局面。[1]

② 军事对峙：曹操大军在长江北岸布防，孙刘联军在江南对峙。周瑜为试探诸葛亮，命其三日内造十万支箭，意图难倒诸葛亮。[1]

③ 事件经过：诸葛亮利用大雾天气，借助鲁肃的船只，装满草人和青布幔，靠近曹营水寨，诱使曹军弓弩手乱箭射来，成功“借”得十万余支箭，解决了箭矢不足的难题。[1]

参考文献：
[1] 《三国演义》第四十六回“用奇谋孔明借箭 献密计黄盖受刑”：
回目概述：本回讲述诸葛亮受周瑜试探，三日内造箭难题；诸葛亮巧用大雾和草船计，成功从曹军处借得十万支箭，展现其神机妙算。
原文摘录：
“肃领命来见孔明。孔明曰：‘吾曾告子敬，休对公瑾说，他必要害我。不想子敬不肯为我隐讳，今日果然又弄出事来。三日内如何造得十万箭？子敬只得救我！’……第三日四更时分，孔明密请鲁肃到船中……是夜大雾漫天，长江之中，雾气更甚……曹寨中，听得擂鼓呐喊……弓弩齐开，箭如雨降……孔明令收船急回……船两边束草上，排满箭枝……孔明教于船上取之，可得十余万枝。”


In [19]:
print(ask("赤壁之战的发生原因与战后格局如何？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s]


① 赤壁之战的发生原因主要是曹操统一北方后，意图南下征服江东孙权和刘备，扩大其统治范围。曹操挟天子以令诸侯，威势日盛，威胁江东政权的安全，孙权和刘备为抗衡曹操的强大军力，联合抵抗，形成了三方对峙的局面。[1]

② 战争爆发的直接导火索是曹操率领大军南下，企图通过水陆并进的方式快速攻占江东。孙权在权衡利弊后，决定联合刘备，共同抵御曹操的进攻。周瑜等吴将积极备战，筑坞防守，准备迎战曹操的水军。[2]

③ 赤壁之战的结果是曹操大败，水军被周瑜火攻烧毁，曹操被迫撤退，江东和荆州暂时得以保全。此战奠定了三国鼎立的基础，孙权稳固江东，刘备得以扩展荆州，形成魏、蜀、吴三国鼎立的局面，影响了中国历史的走向。[3]

参考文献：
[1] 《三国演义》第七十七回“火烧赤壁”：
回目概述：讲述曹操挟天子南征，孙刘联军抗击曹军，赤壁大战爆发的背景。
原文摘录：“曹操挟天子以征四方，威赫中原……孙权大怒，商议起军攻取荆州……忽报曹操起军四十万来报赤壁之仇。”

[2] 《三国演义》第七十八回“周瑜怒杀程普”：
回目概述：孙权命令筑濡须坞防御曹操，周瑜等将领积极备战。
原文摘录：“权曰：‘人无远虑，必有近忧。子明之见甚远。’便差军数万筑濡须坞。晓夜并工，刻期告竣。”

[3] 《三国演义》第七十九回“火烧连营”：
回目概述：赤壁大战曹操大败，周瑜火攻破曹军，三国鼎立格局初现。
原文摘录：“三江水战，赤壁鏖兵。曹军着枪中箭、火焚水溺者，不计其数……赤壁遗雄烈，青年有俊声……巴丘终命处，凭吊欲伤情。”


In [20]:
print(ask("孙悟空为何第一次被压于五行山下？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  3.13it/s]


① 孙悟空大闹天宫，犯了“诳上之罪”，即违抗天庭法令，扰乱天宫秩序，触犯了天条，因此被如来佛祖镇压。[1]

② 孙悟空因其桀骜不驯，曾多次挑战天庭权威，甚至自称“齐天大圣”，引起天庭震怒，最终如来佛祖用法力将其压于五行山下，以示惩戒和约束。[1]

③ 五行山是如来佛祖用五指化作的五座联山，轻轻压住孙悟空，防止其再行作乱，同时也是对其未来皈依佛法的考验和约束。[1]

参考文献：  
[1] 《西游记》第十四回“心猿归正，五行山下定风波”：  
回目概述：讲述孙悟空大闹天宫后，被如来佛祖镇压于五行山下，等待有缘人解救。  
原文摘录：  
“好大圣急纵身又要跳出被佛祖翻掌一扑把这猴王推出西天门外将五指化作金、木、水、火、土五座联山轻轻的把他压住。”  
“当年卵化学为人立志修行果道真。万劫无移居胜境一朝有变散精神。欺天罔上思高位凌圣偷丹乱大伦。恶贯满盈今有报不知何日得翻身。”


In [21]:
print(ask("曹雪芹借“金陵十二钗”群像想要传达哪些女性命运与时代主题？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


① 女性命运的多样性与悲剧性：《红楼梦》中的“金陵十二钗”不仅是美貌女子的集合，更是各具性格、命运迥异的女性群像。她们或富贵或贫贱，或温柔或刚烈，命运多舛，体现了封建社会女性在家族兴衰、社会变迁中的无奈与挣扎。[1]

② 时代背景下的女性困境：这些女性大多生于末世运偏消的时代，身处家族衰败、社会动荡的环境中，虽有才情与美貌，却难逃命运的捉弄和社会的压迫，反映了封建礼教对女性的束缚和时代变迁带来的冲击。[2]

③ 情感与理想的矛盾冲突：金陵十二钗中的女性多情而多感，内心丰富却常被现实所限制。她们的爱情、友情和亲情交织，既有温柔和顺，也有反抗和挣扎，表现了个人情感与社会伦理、家族责任之间的矛盾。[3]

④ 通过群像展现社会风貌与人性复杂：曹雪芹借十二钗的不同命运，折射出当时社会的风气、人情冷暖以及人性的善恶美丑，既有对美好品德的歌颂，也有对虚伪奸佞的批判，体现了深刻的社会批判精神和人文关怀。[4]

参考文献：
[1] 《红楼梦》“金陵十二钗正册”及“又副册”章节：
回目概述：描绘十二位主要女性形象，揭示其各自的性格与命运悲欢。
原文摘录：“满纸荒唐言，一把辛酸泪！都云作者痴，谁解其中味？”  
[2] 《红楼梦》“又副册”中关于女性命运的诗句：
“才自精明志自高，生于末世运偏消。”  
“富贵又何为，襁褓之间父母违。”  
[3] 《红楼梦》关于金桂与香菱的对话及描写：
“宝钗久察其不轨之心，每随机应变，暗以言语弹压其志。”  
“香菱皆答忘记，金桂便不悦，说有意欺瞒了他。”  
[4] 《红楼梦》对女性群像的社会批判与人性描写：
“虽其中大旨谈情，亦不过实录其事，又非假拟妄称，一味淫邀艳约，私订偷盟之可比．因毫不干涉时世，方从头至尾抄录回来，问世传奇。”


In [22]:
print(ask("梁山泊的一百零八将是如何逐步聚集并结为兄弟的？", []) )

Compute Scores: 100%|██████████| 1/1 [00:00<00:00,  2.89it/s]


① 初期聚集：梁山泊最初是由晁盖等人起义聚集，晁盖被推举为山寨首领，吴用为军师，公孙胜掌军权。各路英雄好汉陆续投奔梁山，形成了最初的骨干力量。[1]

② 结义誓盟：宋江等一百零八位好汉在梁山忠义堂上举行盛大聚义，焚香誓盟，立下“替天行道，保境安民”的大义誓言，誓死同生共死，忠义相扶，形成坚不可摧的兄弟情谊。[2]

③ 组织完善与扩充：晁盖等头领安排寨内事务，修筑寨栅，整顿军备，招募更多英雄好汉，逐步扩大队伍规模，形成一百零八将的完整阵容，涵盖各行各业，南北东西，贵贱不分，真正实现了“异姓一家”的兄弟情谊。[3]

参考文献：  
[1] 《水浒传》第三回“史大郎夜走华阴县”及第四回“林教头风雪山神庙”  
原文摘录：晁盖等人起义，聚集梁山，推举晁盖为首领，吴用为军师，公孙胜掌军权。  
[2] 《水浒传》第七回“林冲雪夜上梁山”  
原文摘录：宋江等一百零八人于梁山忠义堂上焚香誓盟，立下替天行道的誓言，誓死同生共死。  
[3] 《水浒传》第八回“李逵闹江州”  
原文摘录：晁盖等头领安排寨内事务，修筑寨栅，整顿军备，招募英雄，形成一百零八将的完整阵容。


In [23]:
# 以下是 20 条针对中国古典四大名著（《红楼梦》《西游记》《水浒传》《三国演义》）的测试问题，每部 5 条，方便您检验系统：
#
# 《红楼梦》测试问题
# 	1.	《红楼梦》的前八十回与后四十回分别聚焦哪些人物命运与家族兴衰？
# 	2.	贾宝玉与林黛玉的爱情悲剧反映了怎样的社会现实与家族压力？
# 	3.	薛宝钗在大观园中扮演了怎样的“理性”与“和事佬”角色？
# 	4.	“大观园”在作品中象征着怎样的繁华与虚幻？
# 	5.	曹雪芹借“金陵十二钗”群像想要传达哪些女性命运与时代主题？
#
# 《西游记》测试问题  ￼
# 	1.	《西游记》的公认作者通常是谁？  ￼
# 	2.	孙悟空为何第一次被压于五行山下？  ￼
# 	3.	唐僧南天竺取经的核心目的是什么？  ￼
# 	4.	“取经四人组”由哪些人物组成？  ￼
# 	5.	《西游记》属于哪种小说体裁？  ￼
#
# 《水浒传》测试问题
# 	1.	《水浒传》的作者一般被认为是谁？
# 	2.	梁山泊的一百零八将是如何逐步聚集并结为兄弟的？
# 	3.	宋江在梁山好汉中被称作什么绰号？
# 	4.	“替天行道”这一口号在小说中体现了怎样的价值观？
# 	5.	《水浒传》通过梁山好汉形象批判了哪些社会矛盾？
#
# 《三国演义》测试问题
# 	1.	《三国演义》的作者是谁？
# 	2.	“草船借箭”发生在什么历史背景和军事对峙中？
# 	3.	桃园三结义象征了怎样的兄弟情谊与忠义精神？
# 	4.	曹操“挟天子以令诸侯”体现了何种政治策略？
# 	5.	赤壁之战的发生原因与战后格局如何？